# Compare All Experiment Results

This notebook provides a comprehensive comparison of all experiments.

**Author:** Gheith Alrawahi  
**Institution:** Nankai University

---


In [25]:
# Cell 1: Imports
import json
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

from config import RESULTS_DIR, ALL_EXPERIMENTS

print(f"Results directory: {RESULTS_DIR}")

# Try to import tabulate, install if not available
try:
    from tabulate import tabulate
except ImportError:
    print("Note: tabulate not installed. Using pandas display instead.")
    tabulate = None

Results directory: d:\Projects\MasterProject\code_v2\results


In [26]:
# Cell 2: Load All Results (Updated for new structure)
def find_experiment_dirs():
    """Find all experiment directories including timestamped ones."""
    experiments = {}
    
    for exp_dir in RESULTS_DIR.iterdir():
        if not exp_dir.is_dir():
            continue
        
        # Check if it has final_results.json
        final_path = exp_dir / 'final_results.json'
        if not final_path.exists():
            continue
        
        # Determine base experiment ID
        dir_name = exp_dir.name
        
        # Handle timestamped directories (e.g., v1_baseline_20251207_135825)
        base_id = dir_name
        for known_id in ['v1_baseline', 'v2_enhanced', 'v3_dkd_beta8', 'v3_1_dkd_beta2', 'v4_saturation', 'teacher']:
            if dir_name.startswith(known_id):
                base_id = known_id
                break
        
        # If multiple runs exist, keep the most recent
        if base_id in experiments:
            existing_dir = experiments[base_id]['dir']
            if exp_dir.name > existing_dir.name:  # Newer timestamp
                experiments[base_id] = {'dir': exp_dir, 'name': dir_name}
        else:
            experiments[base_id] = {'dir': exp_dir, 'name': dir_name}
    
    return experiments

def load_all_results():
    """Load final results from all experiments."""
    exp_dirs = find_experiment_dirs()
    results = {}
    
    for base_id, info in exp_dirs.items():
        exp_dir = info['dir']
        final_path = exp_dir / 'final_results.json'
        
        with open(final_path, 'r') as f:
            data = json.load(f)
        
        # Also load config if available
        config_path = exp_dir / 'config.json'
        if config_path.exists():
            with open(config_path, 'r') as f:
                data['config'] = json.load(f)
        
        # Load history summary
        history_path = exp_dir / 'training_history.csv'
        if history_path.exists():
            data['history_available'] = True
            df = pd.read_csv(history_path)
            data['epochs_recorded'] = len(df)
        else:
            data['history_available'] = False
        
        data['dir_name'] = info['name']
        results[base_id] = data
    
    return results

all_results = load_all_results()
print(f"Loaded {len(all_results)} experiments:")
for exp_id, data in all_results.items():
    dir_name = data.get('dir_name', exp_id)
    history = "✓" if data.get('history_available') else "✗"
    print(f"  - {exp_id} ({dir_name}) [history: {history}]")

Loaded 6 experiments:
  - teacher (teacher) [history: ✓]
  - v1_baseline (v1_baseline_20251207_143632) [history: ✓]
  - v2_enhanced (v2_enhanced_20251208_021230) [history: ✓]
  - v3_1_dkd_beta2 (v3_1_dkd_beta2_20251208_132844) [history: ✓]
  - v3_dkd_beta8 (v3_dkd_beta8_20251208_104226) [history: ✓]
  - v4_saturation (v4_saturation_20251208_185409) [history: ✓]


In [27]:
# Cell 3: Create Comparison Table (Updated with Teacher Gap)
def get_value(data, *keys, default=0):
    """Safely get nested value from dict."""
    result = data
    for key in keys:
        if isinstance(result, dict):
            result = result.get(key, default)
        else:
            return default
    return result if result is not None else default

def create_comparison_table(results):
    """Create a detailed comparison table with Teacher Gap column."""
    rows = []
    
    # Define order
    order = ['teacher', 'v1_baseline', 'v2_enhanced', 'v3_dkd_beta8', 'v3_1_dkd_beta2', 'v4_saturation']
    
    # Get teacher accuracy first
    teacher_acc = get_value(results.get('teacher', {}), 'results', 'final_accuracy', default=0)
    
    for exp_id in order:
        if exp_id not in results:
            continue
        
        data = results[exp_id]
        r = data.get('results', {})
        c = data.get('config', {})
        d = c.get('distillation', {})
        a = c.get('augmentation', {})
        
        # Get accuracy (handle both old and new structure)
        accuracy = r.get('final_accuracy', r.get('student_accuracy', 0))
        
        # Determine model type
        model_type = data.get('model_type', 'student' if exp_id != 'teacher' else 'teacher')
        
        # Calculate Teacher Gap for students
        if model_type == 'student' and teacher_acc > 0:
            teacher_gap = teacher_acc - accuracy
        else:
            teacher_gap = None
        
        row = {
            'ID': exp_id,
            'Type': model_type.capitalize(),
            'Method': d.get('method', '-') if model_type == 'student' else '-',
            'AutoAug': 'Y' if a.get('auto_augment') else 'N',
            'Accuracy (%)': accuracy,  # Keep as float for calculations
            'Teacher Gap (%)': teacher_gap,  # New column
            'Best Epoch': r.get('best_epoch', 0),
            'Epochs': r.get('total_epochs', 0),
            'Time (min)': r.get('training_time_minutes', 0),
            'Early Stop': 'Y' if r.get('early_stopped') else 'N'
        }
        
        # Add retention for students
        if model_type == 'student':
            retention = r.get('retention_rate', 0)
            row['Retention (%)'] = retention if retention else 0
        else:
            row['Retention (%)'] = None
        
        rows.append(row)
    
    df = pd.DataFrame(rows)
    return df

comparison_df = create_comparison_table(all_results)

# Create display version with formatting
display_df = comparison_df.copy()
display_df['Accuracy (%)'] = display_df['Accuracy (%)'].apply(lambda x: f"{x:.2f}")
display_df['Teacher Gap (%)'] = display_df['Teacher Gap (%)'].apply(lambda x: f"{x:.2f}" if pd.notna(x) else '-')
display_df['Time (min)'] = display_df['Time (min)'].apply(lambda x: f"{x:.1f}")
display_df['Retention (%)'] = display_df['Retention (%)'].apply(lambda x: f"{x:.1f}" if pd.notna(x) and x > 0 else '-')

print("\n" + "=" * 110)
print("EXPERIMENT COMPARISON (with Teacher Gap)")
print("=" * 110)

if tabulate:
    print(tabulate(display_df, headers='keys', tablefmt='grid', showindex=False))
else:
    print(display_df.to_string(index=False))


EXPERIMENT COMPARISON (with Teacher Gap)
+----------------+---------+-------------+-----------+----------------+-------------------+--------------+----------+--------------+--------------+-----------------+
| ID             | Type    | Method      | AutoAug   |   Accuracy (%) | Teacher Gap (%)   |   Best Epoch |   Epochs |   Time (min) | Early Stop   | Retention (%)   |
+================+=========+=============+===========+================+===================+==============+==========+==============+==============+=================+
| teacher        | Teacher | -           | N         |          76.65 | -                 |          199 |      200 |        389.7 | N            | -               |
+----------------+---------+-------------+-----------+----------------+-------------------+--------------+----------+--------------+--------------+-----------------+
| v1_baseline    | Student | standard_kd | N         |          76.12 | 0.53              |          200 |      200 |        300

In [28]:
# Cell 4: Key Findings (Refined Narrative - Surgical Analysis)
print("\n" + "=" * 80)
print("KEY FINDINGS - SURGICAL ANALYSIS")
print("=" * 80)

# Get teacher accuracy
teacher_acc = 0
if 'teacher' in all_results:
    teacher_acc = get_value(all_results['teacher'], 'results', 'final_accuracy', default=0)
    print(f"\n TEACHER MODEL (Reference):")
    print(f"   Model: EfficientNetV2-L")
    print(f"   Accuracy: {teacher_acc:.2f}%")

# Get all student accuracies
v1_acc = get_value(all_results.get('v1_baseline', {}), 'results', 'final_accuracy', default=0)
v2_acc = get_value(all_results.get('v2_enhanced', {}), 'results', 'final_accuracy', default=0)
v3_acc = get_value(all_results.get('v3_dkd_beta8', {}), 'results', 'final_accuracy', default=0)
v3_1_acc = get_value(all_results.get('v3_1_dkd_beta2', {}), 'results', 'final_accuracy', default=0)
v4_acc = get_value(all_results.get('v4_saturation', {}), 'results', 'final_accuracy', default=0)

# Finding 1: Robustness of Standard KD
print(f"\n" + "-" * 80)
print("🔬 FINDING 1: Robustness of Standard KD")
print("-" * 80)
print(f"""
   Standard KD proved exceptionally robust against strong augmentation.
   
   • Baseline (v1, no AutoAug): {v1_acc:.2f}%
   • Enhanced (v2, with AutoAug): {v2_acc:.2f}%
   • Improvement: +{v2_acc - v1_acc:.2f}%
   
   ➜ The marginal gain (+{v2_acc - v1_acc:.2f}%) despite adding AutoAugment indicates
     that Standard KD is highly DATA-EFFICIENT and STABLE.
""")

# Finding 2: Fragility of High-Beta DKD
print("-" * 80)
print(" FINDING 2: Fragility of High-Beta DKD (Regularization-Distillation Conflict)")
print("-" * 80)
print(f"""
   Experiment v3 (DKD, β=8.0) collapsed dramatically.
   
   • v3 (DKD β=8.0): {v3_acc:.2f}% ← COLLAPSED (Early stopped at epoch 84)
   • v3.1 (DKD β=2.0): {v3_1_acc:.2f}% ← RECOVERED
   • Recovery gain: +{v3_1_acc - v3_acc:.2f}%
   
   ➜ High reliance on dark knowledge (β=8.0) interferes with strong augmentation noise.
   ➜ DKD requires SENSITIVE TUNING compared to Standard KD.
""")

# Finding 3: Capacity Saturation (Most Important)
print("-" * 80)
print(" FINDING 3: Confirmation of Capacity Saturation (KEY RESULT)")
print("-" * 80)
v2_gap = teacher_acc - v2_acc if teacher_acc > 0 else 0
v4_gap = teacher_acc - v4_acc if teacher_acc > 0 else 0
print(f"""
   The most significant finding: comparison between v2 and v4.
   
   • v2 (Teacher: 76.65%): Student achieved {v2_acc:.2f}% | Gap: {v2_gap:.2f}%
   • v4 (Teacher: 82.54%): Student achieved {v4_acc:.2f}% | Gap: {v4_gap:.2f}%
   
   Despite using a STRONGER teacher in v4 (+5.89% better), the student accuracy
   remained EXACTLY IDENTICAL ({v2_acc:.2f}% = {v4_acc:.2f}%).
   
   ➜ This exact match provides DEFINITIVE EMPIRICAL EVIDENCE that the student
     model (EfficientNetV2-S) has hit its REPRESENTATIONAL CEILING on CIFAR-100.
   ➜ The tiny gap ({v2_gap:.2f}%) proves we are at maximum capacity.
""")

# Summary Statistics
print("=" * 80)
print(" SUMMARY STATISTICS")
print("=" * 80)
print(f"""
   Best Student Accuracy:  {max(v1_acc, v2_acc, v3_acc, v3_1_acc, v4_acc):.2f}%
   Best Retention Rate:    {max(v1_acc, v2_acc, v4_acc)/teacher_acc*100:.2f}%
   Smallest Teacher Gap:   {min(v2_gap, v4_gap):.2f}%
   
   Standard KD vs DKD Winner: Standard KD (+{max(v2_acc, v4_acc) - v3_1_acc:.2f}%)
""")


KEY FINDINGS - SURGICAL ANALYSIS

 TEACHER MODEL (Reference):
   Model: EfficientNetV2-L
   Accuracy: 76.65%

--------------------------------------------------------------------------------
🔬 FINDING 1: Robustness of Standard KD
--------------------------------------------------------------------------------

   Standard KD proved exceptionally robust against strong augmentation.
   
   • Baseline (v1, no AutoAug): 76.12%
   • Enhanced (v2, with AutoAug): 76.19%
   • Improvement: +0.07%
   
   ➜ The marginal gain (+0.07%) despite adding AutoAugment indicates
     that Standard KD is highly DATA-EFFICIENT and STABLE.

--------------------------------------------------------------------------------
 FINDING 2: Fragility of High-Beta DKD (Regularization-Distillation Conflict)
--------------------------------------------------------------------------------

   Experiment v3 (DKD, β=8.0) collapsed dramatically.
   
   • v3 (DKD β=8.0): 66.85% ← COLLAPSED (Early stopped at epoch 84)
   • v

In [29]:
# Cell 5: Export Publication-Quality LaTeX Table (Thesis-Ready)
def export_professional_latex_table(df, filename):
    """Export a publication-quality LaTeX table with auto-bolded best results."""
    
    # Prepare data for LaTeX
    latex_df = df[df['Type'] == 'Student'].copy()  # Only students for main comparison
    
    # Get max values for highlighting
    max_accuracy = latex_df['Accuracy (%)'].max()
    max_retention = latex_df['Retention (%)'].max()
    min_gap = latex_df['Teacher Gap (%)'].min()  # Smaller gap is better
    
    # Build LaTeX table body
    latex_body = ""
    
    for _, row in latex_df.iterrows():
        exp_id = row['ID']
        
        # Format experiment ID
        exp_display = exp_id.replace('_', '-')
        
        # Format method name
        method = row['Method']
        if method == 'standard_kd':
            method_display = 'Standard KD'
        elif method == 'dkd':
            if 'beta8' in exp_id:
                method_display = 'DKD ($\\beta=8.0$)'
            elif 'beta2' in exp_id:
                method_display = 'DKD ($\\beta=2.0$)'
            else:
                method_display = 'DKD'
        else:
            method_display = method
        
        # Format accuracy (bold if max)
        acc = row['Accuracy (%)']
        acc_str = f"\\textbf{{{acc:.2f}}}" if acc == max_accuracy else f"{acc:.2f}"
        
        # Format teacher gap (bold if min - smaller is better)
        gap = row['Teacher Gap (%)']
        gap_str = f"\\textbf{{{gap:.2f}}}" if gap == min_gap else f"{gap:.2f}"
        
        # Format retention (bold if max)
        ret = row['Retention (%)']
        ret_str = f"\\textbf{{{ret:.1f}}}" if ret == max_retention else f"{ret:.1f}"
        
        # Format time
        time_str = f"{row['Time (min)']:.1f}"
        
        # AutoAug indicator
        autoaug = "\\checkmark" if row['AutoAug'] == 'Y' else "-"
        
        latex_body += f"    {exp_display} & {method_display} & {autoaug} & {acc_str} & {gap_str} & {ret_str} & {time_str} \\\\\n"
    
    # Complete LaTeX table
    latex_table = f"""% Auto-generated LaTeX table for thesis
% Best results are highlighted in \\textbf{{bold}}

\\begin{{table}}[htbp]
\\centering
\\caption{{Comparison of Knowledge Distillation Methods on CIFAR-100. 
Teacher: EfficientNetV2-L (76.65\\%), Student: EfficientNetV2-S. 
Best results are highlighted in \\textbf{{bold}}.}}
\\label{{tab:kd_comparison}}
\\begin{{tabular}}{{lcccccc}}
\\toprule
\\textbf{{Experiment}} & \\textbf{{Method}} & \\textbf{{AutoAug}} & \\textbf{{Accuracy}} & \\textbf{{Gap}} & \\textbf{{Retention}} & \\textbf{{Time}} \\\\
 & & & \\textbf{{(\\%)}} & \\textbf{{(\\%)}} & \\textbf{{(\\%)}} & \\textbf{{(min)}} \\\\
\\midrule
{latex_body}\\bottomrule
\\end{{tabular}}
\\end{{table}}
"""
    
    # Save to file
    output_path = RESULTS_DIR / filename
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(latex_table)
    
    print(f"✓ Publication-quality LaTeX table saved: {output_path}")
    print("\n" + "=" * 80)
    print("LATEX TABLE PREVIEW:")
    print("=" * 80)
    print(latex_table)
    
    return latex_table

# Generate the professional LaTeX table
latex_output = export_professional_latex_table(comparison_df, 'comparison_table_professional.tex')

# Also save the simple version
display_df.to_csv(RESULTS_DIR / 'comparison_table.csv', index=False)
print(f"\n✓ CSV saved: {RESULTS_DIR / 'comparison_table.csv'}")

✓ Publication-quality LaTeX table saved: d:\Projects\MasterProject\code_v2\results\comparison_table_professional.tex

LATEX TABLE PREVIEW:
% Auto-generated LaTeX table for thesis
% Best results are highlighted in \textbf{bold}

\begin{table}[htbp]
\centering
\caption{Comparison of Knowledge Distillation Methods on CIFAR-100. 
Teacher: EfficientNetV2-L (76.65\%), Student: EfficientNetV2-S. 
Best results are highlighted in \textbf{bold}.}
\label{tab:kd_comparison}
\begin{tabular}{lcccccc}
\toprule
\textbf{Experiment} & \textbf{Method} & \textbf{AutoAug} & \textbf{Accuracy} & \textbf{Gap} & \textbf{Retention} & \textbf{Time} \\
 & & & \textbf{(\%)} & \textbf{(\%)} & \textbf{(\%)} & \textbf{(min)} \\
\midrule
    v1-baseline & Standard KD & - & 76.12 & 0.53 & 99.3 & 300.2 \\
    v2-enhanced & Standard KD & \checkmark & \textbf{76.19} & \textbf{0.46} & \textbf{99.4} & 305.7 \\
    v3-dkd-beta8 & DKD ($\beta=8.0$) & \checkmark & 66.85 & 9.80 & 87.2 & 124.9 \\
    v3-1-dkd-beta2 & DKD ($\beta

In [30]:
# Cell 6: Generate Enhanced Summary Report (with Refined Narrative)
def generate_enhanced_summary_report():
    """Generate a publication-quality markdown summary report with refined narrative."""
    
    # Get key values
    teacher_acc = get_value(all_results.get('teacher', {}), 'results', 'final_accuracy', default=0)
    v1_acc = get_value(all_results.get('v1_baseline', {}), 'results', 'final_accuracy', default=0)
    v2_acc = get_value(all_results.get('v2_enhanced', {}), 'results', 'final_accuracy', default=0)
    v3_acc = get_value(all_results.get('v3_dkd_beta8', {}), 'results', 'final_accuracy', default=0)
    v3_1_acc = get_value(all_results.get('v3_1_dkd_beta2', {}), 'results', 'final_accuracy', default=0)
    v4_acc = get_value(all_results.get('v4_saturation', {}), 'results', 'final_accuracy', default=0)
    
    report = []
    report.append("# Experiment Results Summary\n")
    report.append(f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")
    
    report.append("\n## Overview\n")
    report.append(f"- **Total experiments:** {len(all_results)}")
    report.append(f"- **Teacher model:** EfficientNetV2-L ({teacher_acc:.2f}%)")
    report.append(f"- **Student model:** EfficientNetV2-S")
    report.append(f"- **Dataset:** CIFAR-100")
    report.append(f"- **Training epochs:** 200 (with early stopping)\n")
    
    report.append("\n## Results Table\n")
    report.append(display_df.to_markdown(index=False))
    report.append("\n")
    
    # Key Findings with Refined Narrative
    report.append("\n## Key Findings\n")
    
    report.append("\n### 1. Robustness of Standard KD\n")
    report.append(f"""Standard KD proved exceptionally robust against strong augmentation. 
The baseline (v1) achieved {v1_acc:.2f}%, and the enhanced version (v2) reached {v2_acc:.2f}%. 
The marginal gain (+{v2_acc - v1_acc:.2f}%) despite adding AutoAugment indicates that 
**Standard KD is highly data-efficient** and stable.\n""")
    
    report.append("\n### 2. Fragility of High-Beta DKD\n")
    report.append(f"""Experiment v3 (DKD, β=8.0) collapsed to {v3_acc:.2f}%, confirming the 
**Regularization-Distillation Conflict**. High reliance on dark knowledge interferes with 
strong augmentation noise. Reducing β to 2.0 (v3.1) recovered performance to {v3_1_acc:.2f}%, 
validating that DKD requires sensitive tuning compared to Standard KD.\n""")
    
    report.append("\n### 3. Confirmation of Capacity Saturation (Key Result)\n")
    v2_gap = teacher_acc - v2_acc
    v4_gap = teacher_acc - v4_acc
    report.append(f"""The most significant finding is the comparison between v2 and v4. 
Despite using a stronger teacher in v4 (82.54% vs 76.65%), the student accuracy remained 
**exactly identical ({v2_acc:.2f}% = {v4_acc:.2f}%)**. 

This exact match provides **definitive empirical evidence** that the student model 
(EfficientNetV2-S) has hit its **representational ceiling** on this dataset.

- **Teacher Gap (v2):** {v2_gap:.2f}%
- **Teacher Gap (v4):** {v4_gap:.2f}%

The tiny gap of less than 0.5% proves we are operating at maximum student capacity.\n""")
    
    report.append("\n## Summary Statistics\n")
    best_acc = max(v1_acc, v2_acc, v3_acc, v3_1_acc, v4_acc)
    best_retention = best_acc / teacher_acc * 100 if teacher_acc > 0 else 0
    report.append(f"| Metric | Value |")
    report.append(f"|--------|-------|")
    report.append(f"| Best Student Accuracy | {best_acc:.2f}% |")
    report.append(f"| Best Retention Rate | {best_retention:.2f}% |")
    report.append(f"| Smallest Teacher Gap | {min(v2_gap, v4_gap):.2f}% |")
    report.append(f"| Standard KD vs DKD | Standard KD wins by +{best_acc - v3_1_acc:.2f}% |")
    report.append("")
    
    report.append("\n## Experiment Details\n")
    for exp_id in ['teacher', 'v1_baseline', 'v2_enhanced', 'v3_dkd_beta8', 'v3_1_dkd_beta2', 'v4_saturation']:
        if exp_id not in all_results:
            continue
        data = all_results[exp_id]
        results = data.get('results', {})
        acc = get_value(data, 'results', 'final_accuracy', default=0)
        report.append(f"\n### {exp_id}")
        report.append(f"- **Accuracy:** {acc:.2f}%")
        report.append(f"- **Best Epoch:** {results.get('best_epoch', 'N/A')}")
        report.append(f"- **Training Time:** {results.get('training_time_minutes', 0):.1f} minutes")
        if exp_id != 'teacher' and teacher_acc > 0:
            gap = teacher_acc - acc
            report.append(f"- **Teacher Gap:** {gap:.2f}%")
        report.append("")
    
    # Write report
    report_text = '\n'.join(report)
    report_path = RESULTS_DIR / 'summary_report.md'
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(report_text)
    
    print(f"✓ Enhanced summary report saved: {report_path}")
    return report_text

report = generate_enhanced_summary_report()
print("\n" + "=" * 80)
print(report)

✓ Enhanced summary report saved: d:\Projects\MasterProject\code_v2\results\summary_report.md

# Experiment Results Summary

**Generated:** 2025-12-09 00:26


## Overview

- **Total experiments:** 6
- **Teacher model:** EfficientNetV2-L (76.65%)
- **Student model:** EfficientNetV2-S
- **Dataset:** CIFAR-100
- **Training epochs:** 200 (with early stopping)


## Results Table

| ID             | Type    | Method      | AutoAug   |   Accuracy (%) | Teacher Gap (%)   |   Best Epoch |   Epochs |   Time (min) | Early Stop   | Retention (%)   |
|:---------------|:--------|:------------|:----------|---------------:|:------------------|-------------:|---------:|-------------:|:-------------|:----------------|
| teacher        | Teacher | -           | N         |          76.65 | -                 |          199 |      200 |        389.7 | N            | -               |
| v1_baseline    | Student | standard_kd | N         |          76.12 | 0.53              |          200 |      200 |        